In [1]:
#import the needed libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore') #Supress unnecessary warnings for readability and cleaner presentation
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
from IPython.display import display
import pickle
color = sns.color_palette()

import pickle
pd.set_option('display.float_format', lambda x: '%.3f' % x) #Limiting floats output to 3 decimal points
import gc
print (gc.isenabled())

True


In [2]:
#deeplearning
import tensorflow
import keras
from keras.models import Model,Sequential

from keras.layers.core import Dense, Reshape, Lambda,Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Embedding, merge
from keras import backend as K

Using TensorFlow backend.


In [3]:
# concatenate all product-ids into a single string
# thanks to https://www.kaggle.com/eoakley/start-here-simple-submission

def products_concat(series):
    out = ''
    for product in series:
           out = out + str(int(product)) + ' '
    
    if out != '':
        return out.rstrip()
    else:
        return 'None'

In [4]:
def df_split(df):
    df_reordered = df[df['reordered'] == 1]
    df_reordered = df_reordered.groupby('order_id')['product_id'].apply(products_concat)
    
    try:
        df_reordered = df_reordered.reset_index()
        df_reordered.columns = ['order_id','products_list']
    except:
        df_reordered = df_reordered.reset_index(level = ['order_id'])
        df_reordered.columns = ['order_id','products_list']
        
    return df_reordered

In [5]:
def multilabel_fscore(ytrue, ypred):
    from keras import backend as K
    """
    ex1:
    y_true = [1, 2, 3]
    y_pred = [2, 3]
    return: 0.8
    
    ex2:
    y_true = ["None"]
    y_pred = [2, "None"]
    return: 0.666
    
    ex3:
    y_true = [4, 5, 6, 7]
    y_pred = [2, 4, 8, 9]
    return: 0.25
    
    """
    #ytrue = K.eval(ytrue)
    #ypred = K.eval(ypred)
    F1_score = []
    y_pred = []
    y_true = []
    true = list(ytrue) 
    pred = list(ypred)
    for items in range(len(true)):
        y_true = true[items]
        y_pred = pred[items]
        correct = sum([1 for i in y_pred if i in y_true])
        if correct > 0:

            precision = correct / len(y_pred)

            recall =    correct / len(y_true)

            F1_score.append((2 * precision * recall) / (precision + recall))

        else:
            F1_score.append(0)

    return np.mean(F1_score)

if __name__ == '__main__':

    #print(multilabel_fscore([[2,3],['None']], [[2,3,4],['None']]))
    print(multilabel_fscore([('None',),('2','3',)], [('None',),('2','3','4',)]))

0.9


In [ ]:
#Now let's get and put the data in  pandas dataframe
order_products_train = pd.read_csv('/training/order_products__train.csv')
order_products_prior = pd.read_csv('/training/order_products__prior.csv')
orders = pd.read_csv('/training/orders.csv',engine ='c')
#aisles = pd.read_csv('input/aisles.csv')
#departments = pd.read_csv('input/departments.csv')
display(order_products_prior.head(1))
display(orders.head(1))

In [ ]:
products = pd.read_csv('/training/products.csv')
value = list(products['product_id'].astype(str))
value.append('None')
#save data
pickle.dump(value, open('prod_dict.p', 'wb'), protocol=4)

In [ ]:
#create one hot encode of aisle and dept
df_additional = products[['product_id','aisle_id','department_id']]
df_additional[['aisle_id','department_id']] = df_additional[['aisle_id','department_id']].astype(str)
df_additional = pd.get_dummies(df_additional, columns=['aisle_id','department_id'])
display(df_additional.head(2))

In [ ]:
prior_reo = order_products_prior[order_products_prior['reordered'] == 1].drop(['reordered','add_to_cart_order'], axis=1)
display(prior_reo.head(1))
train_reo = order_products_train[order_products_train['reordered'] == 1].drop(['reordered','add_to_cart_order'], axis=1)
prior_reo = prior_reo.merge(train_reo,on=['order_id','product_id'], how = 'outer')
display(prior_reo.head(1))
prior_reo = prior_reo.merge(df_additional, on='product_id', how='left')
print('joined')
prior_reo = prior_reo.drop(['product_id'], axis =1)
print('merging')
prior_reo = prior_reo.merge(orders[['user_id','order_number','order_id']],on='order_id', how='left')
prior_reo = prior_reo.sort_values(['user_id','order_number'])
print (prior_reo.info())
print('aggregating')
prior_reo = prior_reo.groupby(['user_id','order_number', 'order_id']).sum()
display(prior_reo.head(5))
print('shifting previous orders')
prior_reo= prior_reo.groupby(level=0).shift(1)
#apply(lambda x: pd.rolling_sum(x, window=2, min_periods=0)
prior_reo = prior_reo.reset_index()
display(prior_reo.head(5))
#print('saving')
#prior_reo.to_csv('input/prior_prods.csv', index=False)
#display(prior_reo.head(2))

In [ ]:
order_prod_list = order_products_train.merge(order_products_prior, how = 'outer')
display(order_prod_list.head(1))
df_prior_prod = df_split(order_prod_list)
display(df_prior_prod.head(1))
df_train = orders.merge(df_prior_prod, on='order_id', how='left')
df_train = df_train.merge(prior_reo, on=['order_id','user_id','order_number'], how='left')
df_train['products_list'] = df_train['products_list'].fillna('None')
df_train = df_train.fillna(0)
display(df_train.tail(2))

In [ ]:
df_eval = df_train[df_train['eval_set'] == 'train']
display(df_eval.head(2))

In [ ]:
del order_products_train,order_products_prior,orders,products,order_prod_list
del df_prior_prod, prior_reo, df_additional
gc.collect()

In [ ]:
df_eval = df_eval.drop(['eval_set','user_id'], axis =1)

df_test =  df_train[df_train['eval_set'] == 'test']
df_train = df_train[df_train['eval_set'] == 'prior']
df_train = df_train.drop(['eval_set','user_id'], axis =1)
display(df_test.head(1))

In [ ]:
#save data
pickle.dump(df_eval, open('eval_orders.p', 'wb'), protocol=4)

pickle.dump(df_train, open('train_orders.p', 'wb'), protocol=4)

pickle.dump(df_test, open('test_orders.p', 'wb'), protocol=4)

## Checkpoint
data can be loaded from here

### new features ideas
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float16)
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)

In [6]:
'''
evals = open('eval_orders.p','rb')
df_eval = pickle.load(evals)
evals.close()

train = open('train_orders.p','rb')
df_train = pickle.load(train)
train.close()
'''
df_eval = pd.read_pickle(open('eval_orders.p','rb'))
df_train = pd.read_pickle(open('train_orders.p','rb'))
'''
test = open('test_orders.p','rb')
test_orders = pickle.load(test)
test.close()
'''

"\ntest = open('test_orders.p','rb')\ntest_orders = pickle.load(test)\ntest.close()\n"

In [ ]:
X_train = df_train.drop(['products_list'], axis = 1)
X_eval = df_eval.drop(['products_list'], axis = 1)

### setting up keras

In [ ]:
X_train = X_train.set_index('order_id')
X_eval = X_eval.set_index('order_id')
display(X_train.head(1))

In [12]:
X_train[['order_dow', 'order_hour_of_day']] = X_train[['order_dow', 'order_hour_of_day']].astype(str)
X_train = pd.get_dummies(X_train, columns=['order_dow', 'order_hour_of_day'])
X_train = X_train.astype('int8')
display(X_train.head(1))
X_train = X_train.fillna(-1)
display(X_train.head(1))
X_train = X_train.replace(0,-1)
display(X_train.head(1))

,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1


In [ ]:
X_eval[['order_dow', 'order_hour_of_day']] = X_eval[['order_dow', 'order_hour_of_day']].astype(str)
X_eval = pd.get_dummies(X_eval, columns=['order_dow', 'order_hour_of_day'])
X_eval = X_eval.astype('int8')
display(X_eval.head(1))
X_eval = X_eval.fillna(-1)
display(X_eval.head(1))
X_eval = X_eval.replace(0,-1)
display(X_eval.head(1))

In [13]:
pickle.dump(X_train, open('X_train.p', 'wb'), protocol=4)
pickle.dump(X_eval, open('X_eval.p', 'wb'), protocol=4)

In [7]:
X_train = pickle.load(open('X_train.p','rb'))
X_eval = pickle.load(open('X_eval.p','rb'))
#vectorise the products list
#from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
value = pickle.load( open('/dict/prod_dict.p', 'rb'))
dicts = []
for i in value:
    dicts.append(tuple(i.split()))

vectorizer = MultiLabelBinarizer(sparse_output=True).fit(dicts)

def y_target(df):
    train_target = df['products_list'].values
    print(train_target[0])
    y_train = []
    for i in train_target:
        y_train.append(tuple(i.split()))
    print (y_train[0])
    return y_train

y_train = y_target(df_train)
y_eval = y_target(df_eval)  

y_train = vectorizer.transform(y_train)
y_eval = vectorizer.transform(y_eval)
 
print(y_train.dtype)

196 12427 26088
('None',)
22963 7963 16589 32792 41787 22825 24852 45066 5450 22559 45613 33957
('196', '25133', '38928', '26405', '39657', '10258', '13032', '26088', '49235', '46149')
int64


In [8]:
#deleted df
del df_train, df_eval
gc.collect()
   
print (vectorizer.classes_[49688])
y_train = y_train.astype('int8')
y_eval = y_eval.astype('int8')
print(y_train.dtype)

None
int8


In [9]:
keras_eval = X_eval.as_matrix()
print (keras_eval.shape)

(131209, 188)


In [15]:
#df_order = X_train['order_number']
keras_train = X_train.as_matrix()
print(keras_train.shape)

(3214874, 188)


In [16]:
#min_max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
keras_train = scaler.fit_transform(keras_train)
keras_eval = scaler.transform(keras_eval)

In [17]:
#keras metrics
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.
    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.
    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.
    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [18]:
#batch generator
def batch_generator(X, y, n_batches, batch_size):
    #shuffle data first
    counter=0
    shuffle_index = np.arange(np.shape(X)[0])
    np.random.shuffle(shuffle_index)
    X =  X[shuffle_index]
    y =  y[shuffle_index, :]
    I = 0
    #create batches
    while I ==0:
        index_batch = shuffle_index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X[index_batch]
        y_batch = y[index_batch,:].todense()
        counter += 1
        if counter > n_batches:
            I = 1
        else:
            yield(X_batch,np.array(y_batch))

In [26]:
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
# create model
model = Sequential()
model.add(Dense(216,input_dim=188, activation='relu'))
model.add(BatchNormalization())
# Hidden layers
model.add(Dense(375,activation='relu'))
model.add(Dense(480,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(624,activation='relu'))
model.add(Dense(700,activation='relu'))
model.add(Dense(866,activation='relu'))
model.add(Dense(1024,activation='relu'))
model.add(Dense(1248,activation='relu'))
model.add(Dense(1560,activation='relu'))
model.add(Dense(1600,activation='relu'))
model.add(Dense(1700,activation='relu'))
model.add(Dense(1720,activation='relu'))
model.add(Dense(1740,activation='relu'))
model.add(Dense(1760,activation='relu'))
model.add(Dropout(0.30))
model.add(Dense(1800,activation='relu'))
model.add(Dense(1820,activation='relu'))
model.add(Dense(1800,activation='relu'))
model.add(Dense(1780,activation='relu'))
model.add(Dense(1750,activation='relu'))
model.add(Dense(1600,activation='relu'))
model.add(Dense(1580,activation='relu'))
model.add(Dense(1550,activation='relu'))
model.add(Dense(1500,activation='relu'))
model.add(Dense(1490,activation='relu'))
model.add(Dense(1470,activation='relu'))
model.add(Dense(1420,activation='relu'))
model.add(Dense(1400,activation='relu'))
model.add(Dropout(0.30))
model.add(Dense(1380,activation='relu'))
model.add(Dense(1350,activation='relu'))
model.add(Dense(1300,activation='relu'))

#output layer
model.add(Dense(49689, activation='sigmoid'))
try:
    #load weights
    #model = load_model('/model/my_model.hdf5')
    #model.load_weights("/model/weights.best.hdf5")
    print('loaded on weights')
except:
    print('load weights failed')
    #model.load_weights("/model/weights.best.hdf5")
    #model = load_model('my_model.hdf5')
#adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.99, epsilon=1e-03, decay=0.0001)
#RMSprop = keras.optimizers.RMSprop(lr=0.001)
#Nadam = keras.optimizers.Nadam(lr=0.001)
Adagrad = keras.optimizers.Adagrad(lr=0.001)


def binary_crossentropy(y_true, y_pred):
    from keras import backend as K
    return K.sum(K.binary_crossentropy(y_pred, y_true), axis=-1)
    
#NUM_CLASSES =49689
model.compile(loss=binary_crossentropy,optimizer= Adagrad, metrics = [fbeta_score])
# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
#earlyStopping= keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=1, mode='min')
callbacks_list = [checkpoint]
keras_train = np.array(X_train)
display(X_train.head(1))

loaded on weights


,order_number,days_since_prior_order,aisle_id_1,aisle_id_10,aisle_id_100,aisle_id_101,aisle_id_102,aisle_id_103,aisle_id_104,aisle_id_105,...,order_hour_of_day_21,order_hour_of_day_22,order_hour_of_day_23,order_hour_of_day_3,order_hour_of_day_4,order_hour_of_day_5,order_hour_of_day_6,order_hour_of_day_7,order_hour_of_day_8,order_hour_of_day_9
order_id,,,,,,,,,,,,,,,,,,,,,
2539329,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,1,-1


In [ ]:
print('Training')

batch_size = 256
epochs = 1
n_batches = 200

#flag to train with all training data
train_with_all = 0

if train_with_all == 1:
    tot_training = keras_train.shape[0]
    n_batches = int(tot_training / batch_size)
    print('tot number of batches: {}'.format(n_batches))
else:
    tot_training = min(batch_size*n_batches,keras_train.shape[0])
    
print('tot items to train: {}'.format(tot_training))
    
# Training cycle
for epoch in range(epochs):
    # Loop over all batches
        print('starting epoch {}'.format(epoch+1))
        for batch_features, batch_labels in batch_generator(keras_train, y_train,n_batches,batch_size):
            model.fit(batch_features, batch_labels,batch_size=batch_size,epochs=200,verbose=0, validation_split=0.05,callbacks= callbacks_list, shuffle='false')
            print(model.evaluate(batch_features,batch_labels, verbose=0))
model.save("my_model.hdf5")

Training
tot items to train: 51200
starting epoch 1
[94.950597763061523, 0.0]
[98.476428031921387, 0.0]
[112.32791328430176, 0.0]
[101.183762550354, 0.0]
[103.2614860534668, 0.0]
[98.980120658874512, 0.0]
[101.93929862976074, 0.0]
[94.132098197937012, 0.0]


In [ ]:
def pred_class(y_eval,keras_eval):
        from scipy.sparse import csr_matrix
        print('Predicting')
        predicted_output = model.predict(keras_eval)
        predicted_output[predicted_output>=0.1] = 1
        predicted_output[predicted_output<0.1] = 0
        pred = csr_matrix(predicted_output)
        true = csr_matrix(y_eval)
        y_eval_pred = vectorizer.inverse_transform(pred)
        y_eval_true = vectorizer.inverse_transform(true)
        print('true eval set values {}'.format(y_eval_true[:2]))
        print('pred eval set values {}\n'.format(y_eval_pred[:2]))
        print (multilabel_fscore(y_eval_true,y_eval_pred))
        
pred_class(y_eval,keras_eval)

Predicting
true eval set values [('10258', '13032', '196', '25133', '26088', '26405', '38928', '39657', '46149', '49235'), ('16589', '22559', '22825', '22963', '24852', '32792', '33957', '41787', '45066', '45613', '5450', '7963')]
pred eval set values [(), ()]

0.0
